In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np

# Exploring
import scipy.stats as stats

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

import acquire
import summarize
import prepare
import env

#### Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

###### Be sure to do the correct join. We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid
###### only include properties with a transaction in 2017, and include only the last transaction for each properity (so no duplicate property id's), along with zestimate error and date of transaction.
##### only include properties that include a latitude and longitude value

In [5]:
df=acquire.get_zillow_data()
df=pd.DataFrame(df)

In [9]:
summarize.df_summary(df)

--- Shape: (77381, 62)
--- Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 62 columns):
parcelid                        77381 non-null int64
logerror                        77381 non-null float64
transactiondate                 77381 non-null object
id                              77381 non-null int64
parcelid                        77381 non-null int64
airconditioningtypeid           24953 non-null float64
architecturalstyletypeid        206 non-null float64
basementsqft                    50 non-null float64
bathroomcnt                     77381 non-null float64
bedroomcnt                      77381 non-null float64
buildingclasstypeid             15 non-null float64
buildingqualitytypeid           49672 non-null float64
calculatedbathnbr               76772 non-null float64
decktypeid                      614 non-null float64
finishedfloor1squarefeet        6023 non-null float64
calculatedfinishedsquarefeet    77185 non-null float

   num_cols_missing    pct_cols_missing  num_rows
0                19   30.64516129032258         2
1                20   32.25806451612903        10
2                21   33.87096774193548        83
3                22  35.483870967741936       262
4                23  37.096774193548384       599
5                24   38.70967741935484       660
6                25   40.32258064516129      5768
7                26  41.935483870967744      7996
8                27   43.54838709677419     12410
9                28   45.16129032258064     20660
10               29  46.774193548387096     16098
11               30   48.38709677419355      6391
12               31                50.0      3604
13               32   51.61290322580645      2089
14               33    53.2258064516129       300
15               34   54.83870967741935       230
16               35  56.451612903225815        29
17               36   58.06451612903226        23
18               37   59.67741935483871        28


AttributeError: 'DataFrame' object has no attribute 'dtype'

summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 62 columns):
parcelid                        77381 non-null int64
logerror                        77381 non-null float64
transactiondate                 77381 non-null object
id                              77381 non-null int64
parcelid                        77381 non-null int64
airconditioningtypeid           24953 non-null float64
architecturalstyletypeid        206 non-null float64
basementsqft                    50 non-null float64
bathroomcnt                     77381 non-null float64
bedroomcnt                      77381 non-null float64
buildingclasstypeid             15 non-null float64
buildingqualitytypeid           49672 non-null float64
calculatedbathnbr               76772 non-null float64
decktypeid                      614 non-null float64
finishedfloor1squarefeet        6023 non-null float64
calculatedfinishedsquarefeet    77185 non-null float64
finishedsquarefeet12         

In [14]:
df.shape


(77381, 62)

In [15]:
df.dtypes

parcelid                          int64
logerror                        float64
transactiondate                  object
id                                int64
parcelid                          int64
airconditioningtypeid           float64
architecturalstyletypeid        float64
basementsqft                    float64
bathroomcnt                     float64
bedroomcnt                      float64
buildingclasstypeid             float64
buildingqualitytypeid           float64
calculatedbathnbr               float64
decktypeid                      float64
finishedfloor1squarefeet        float64
calculatedfinishedsquarefeet    float64
finishedsquarefeet12            float64
finishedsquarefeet13            float64
finishedsquarefeet15            float64
finishedsquarefeet50            float64
finishedsquarefeet6             float64
fips                            float64
fireplacecnt                    float64
fullbathcnt                     float64
garagecarcnt                    float64


In [16]:
df.describe()

,parcelid,logerror,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,...,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock
count,"77,381.00","77,381.00","77,381.00","77,381.00","24,953.00",206.00,50.00,"77,381.00","77,381.00",15.00,...,"77,118.00","17,560.00",172.00,"77,269.00","77,380.00","77,381.00","77,379.00","77,376.00","2,886.00","77,137.00"
mean,"13,007,150.51",0.02,"1,495,138.90","13,007,150.51",1.81,7.39,679.72,2.30,3.05,3.93,...,"1,968.64",1.43,1.00,"189,314.53","490,134.48","2,016.00","301,095.41","5,995.53",14.09,"60,496,733,646,264.13"
std,"3,481,345.65",0.17,"860,907.05","3,481,345.65",2.97,2.73,689.70,1.00,1.14,0.26,...,23.79,0.54,0.00,"230,087.42","653,443.97",0.00,"492,596.03","7,622.84",2.19,"1,535,241,981,138.57"
min,"10,711,855.00",-4.66,349.00,"10,711,855.00",1.00,2.00,38.00,0.00,0.00,3.00,...,"1,824.00",1.00,1.00,44.00,"1,000.00","2,016.00",161.00,19.92,3.00,"60,371,011,101,000.00"
25%,"11,538,305.00",-0.02,"752,070.00","11,538,305.00",1.00,7.00,273.00,2.00,2.00,4.00,...,"1,953.00",1.00,1.00,"84,265.00","207,000.00","2,016.00","85,504.00","2,715.63",14.00,"60,373,109,005,002.00"
50%,"12,531,568.00",0.01,"1,497,932.00","12,531,568.00",1.00,7.00,515.00,2.00,3.00,4.00,...,"1,970.00",1.00,1.00,"136,499.00","358,975.50","2,016.00","203,372.00","4,450.69",15.00,"60,376,032,003,008.00"
75%,"14,211,831.00",0.04,"2,240,535.00","14,211,831.00",1.00,7.00,796.50,3.00,4.00,4.00,...,"1,987.00",2.00,1.00,"218,787.00","569,001.50","2,016.00","366,796.50","6,927.79",15.00,"60,590,423,251,008.00"
max,"167,689,317.00",5.26,"2,982,274.00","167,689,317.00",13.00,21.00,"3,560.00",18.00,16.00,4.00,...,"2,016.00",6.00,1.00,"11,421,790.00","49,061,236.00","2,016.00","48,952,198.00","586,639.30",99.00,"483,030,105,084,015.00"


In [22]:
pd.DataFrame(df.columns)

,0
0,parcelid
1,logerror
2,transactiondate
3,id
4,parcelid
5,airconditioningtypeid
6,architecturalstyletypeid
7,basementsqft
8,bathroomcnt
9,bedroomcnt


In [23]:
df.shape

(77381, 62)

In [29]:
numbers_missing=df.isnull().sum()

3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [30]:
missing = (df.isnull().sum())/df.shape[0]

In [28]:
summarize.nulls_by_col(df)

,num_rows_missing,pct_rows_missing
parcelid,0,0.00
logerror,0,0.00
transactiondate,0,0.00
id,0,0.00
parcelid,0,0.00
airconditioningtypeid,52428,0.68
architecturalstyletypeid,77175,1.00
basementsqft,77331,1.00
bathroomcnt,0,0.00
bedroomcnt,0,0.00


In [31]:

pct_missing = (df.isnull().sum())/df.shape[0]

In [33]:
pct_missing

parcelid                                       0.00
logerror                                       0.00
transactiondate                                0.00
id                                             0.00
parcelid                                       0.00
airconditioningtypeid                          0.68
architecturalstyletypeid                       1.00
basementsqft                                   1.00
bathroomcnt                                    0.00
bedroomcnt                                     0.00
buildingclasstypeid                            1.00
buildingqualitytypeid                          0.36
calculatedbathnbr                              0.01
decktypeid                                     0.99
finishedfloor1squarefeet                       0.92
calculatedfinishedsquarefeet                   0.00
finishedsquarefeet12                           0.05
finishedsquarefeet13                           1.00
finishedsquarefeet15                           0.96
finishedsqua

AttributeError: 'DataFrame' object has no attribute 'desrcibe'